Spark end-to-end example

Using this file complete the End-to-End example in Chapter 1 of textbook.

Try different operations using DataFrame functions and SQL.

In [1]:
import os
from pyspark.sql import SparkSession

# Set Java path directly (replace this with your actual Java path)
os.environ["JAVA_HOME"] = r"C:\Program Files\Microsoft\jdk-11.0.26.4-hotspot"

# Create a Spark session
spark = SparkSession.builder.appName("Flight Data Analysis").getOrCreate()

# File path
file_path = "D:/python/Week4/2012-summary.csv"

# Read the CSV file
flightData2012 = spark.read \
    .option("inferSchema", "true") \
    .option("header", "true") \
    .csv(file_path)

# Print the schema
flightData2012.printSchema()

# Show the first few rows
flightData2012.show()


root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: integer (nullable = true)

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Croatia|    1|
|       United States|            Ireland|  252|
|               Egypt|      United States|   13|
|       United States|              India|   62|
|       United States|          Singapore|   25|
|       United States|            Grenada|   46|
|          Costa Rica|      United States|  522|
|             Senegal|      United States|   31|
|              Guyana|      United States|   65|
|       United States|   Marshall Islands|   30|
|       United States|       Sint Maarten|  245|
|             Bolivia|      United States|   35|
|            Anguilla|      United States|   19|
|       United States|           Paraguay|    5|
|       United States| 

In [2]:
#Group by a column (e.g., "ORIGIN_COUNTRY_NAME") and sum the counts:
flightData2012.groupBy("ORIGIN_COUNTRY_NAME").sum("count").show()


+--------------------+----------+
| ORIGIN_COUNTRY_NAME|sum(count)|
+--------------------+----------+
|            Paraguay|         5|
|              Russia|       148|
|            Anguilla|        19|
|             Senegal|        33|
|              Sweden|        54|
|            Kiribati|        26|
|              Guyana|        79|
|         Philippines|       112|
|           Singapore|        25|
|                Fiji|        26|
|              Turkey|        89|
|             Germany|      1413|
|         Afghanistan|         5|
|            Cambodia|         1|
|              Jordan|        43|
|               Palau|        35|
|Turks and Caicos ...|       183|
|              France|       840|
|              Greece|        21|
|British Virgin Is...|        78|
+--------------------+----------+
only showing top 20 rows



Using SQL for the same operations

In [3]:
#register the dataframe as sql view
flightData2012.createOrReplaceTempView("flights")


In [4]:
#Run SQL queries on the view:
# Example query to fetch top 10 records
spark.sql("SELECT * FROM flights LIMIT 10").show()



+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|            Ireland|  252|
|            Egypt|      United States|   13|
|    United States|              India|   62|
|    United States|          Singapore|   25|
|    United States|            Grenada|   46|
|       Costa Rica|      United States|  522|
|          Senegal|      United States|   31|
|           Guyana|      United States|   65|
|    United States|   Marshall Islands|   30|
+-----------------+-------------------+-----+



In [5]:
# Group by and aggregate
spark.sql("""
    SELECT ORIGIN_COUNTRY_NAME, SUM(count) as total_count
    FROM flights
    GROUP BY ORIGIN_COUNTRY_NAME
    ORDER BY total_count DESC
""").show()

+-------------------+-----------+
|ORIGIN_COUNTRY_NAME|total_count|
+-------------------+-----------+
|      United States|     385290|
|             Canada|       8097|
|             Mexico|       5974|
|     United Kingdom|       1713|
|              Japan|       1451|
|            Germany|       1413|
| Dominican Republic|       1123|
|        The Bahamas|       1054|
|             France|        840|
|           Colombia|        806|
|        South Korea|        703|
|              China|        610|
|            Jamaica|        610|
|        Netherlands|        607|
|             Brazil|        567|
|         Costa Rica|        555|
|        El Salvador|        465|
|              Spain|        393|
|             Panama|        393|
|              Italy|        377|
+-------------------+-----------+
only showing top 20 rows



In [8]:
# Filter rows
spark.sql("""
    SELECT * FROM flights
    WHERE count > 10
""").show()

+--------------------+--------------------+-----+
|   DEST_COUNTRY_NAME| ORIGIN_COUNTRY_NAME|count|
+--------------------+--------------------+-----+
|       United States|             Ireland|  252|
|               Egypt|       United States|   13|
|       United States|               India|   62|
|       United States|           Singapore|   25|
|       United States|             Grenada|   46|
|          Costa Rica|       United States|  522|
|             Senegal|       United States|   31|
|              Guyana|       United States|   65|
|       United States|    Marshall Islands|   30|
|       United States|        Sint Maarten|  245|
|             Bolivia|       United States|   35|
|            Anguilla|       United States|   19|
|Turks and Caicos ...|       United States|  183|
|               Italy|       United States|  381|
|            Pakistan|       United States|   12|
|       United States|              Russia|  148|
|       United States|Federated States ...|   63|


In [6]:
spark.stop()